In [3]:
import Gpib
import time
import matplotlib
import matplotlib.pyplot as plt

import ipywidgets as widgets

import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

import numpy as np

%matplotlib inline


In [4]:
class ParamError(Exception):
    pass

class hp4142b(object):
    
    def __init__(self, inst):
        self.mode = "NORM"
        self.inst = inst
        self.reset()
        print(self.idn())
                
    def write(self, cmd):
        if not cmd.endswith("\n"):
            cmd = cmd+"\n"
        #print("CMD: %s" % cmd)
        return inst.write(cmd)

    def idn(self):
        self.write("*IDN?")
        try:
            return(inst.read().decode('ascii'))
        except:
            print(gpib_err(inst.ibsta()))
            return None   
    
    def status(self):
        self.write("*STB?")
        st = self.inst.read().decode('ascii')
        print("Status: %s %s" %(st,  statusdecode(int(st))))
        
    def nub(self):
        self.write("NUB?")
        return self.readresult()
    
    def units(self):
        self.write("UNT?")
        return self.readresult()
    
    def reset(self):
        self.write("*RST")
        self.write("FMT1,1")
    
    def voltage(self, ch, v=0.0, range=0, i_lim=0.001, lim_mode=0):
        return self.write("DV%d,%d,%+.3e,%+.3e,%d\n" % ( ch, range, v, i_lim, lim_mode ))
    
    def pulsed_v(self, ch, base=0.0, pulse=0.0, range=0, i_lim=0.001):
        self.mode="PULSED"
        return self.write("PDV%d,%d,%+.3e,%+.3e,%+.3e"%(ch, range, base, pulse, i_lim))
    
    def pulsed_i(self, ch, base=0.0, pulse=0.0, v_lim=1):
        self.mode="PULSED"
        return self.write("PDI%d,%d,%+.3e,%+.3e,%+.3e"%(ch, range, base, pulse, v_lim))
    
    def current(self, ch, i=0.0, range=0, v_lim=1, lim_mode=0):
        return self.write("DI%d,%d,%+.3e,%+.3e,%d\n"%(ch, range, i, v_lim, lim_mode))

    def sweep_i(self, ch, mode='lin', range=0, start=0, stop=1e-3, n=50, v_lim=1, p_lim=1):
        self.mode="SWEEP"
        mode_n=1;
        if mode=='log':
            mode_n=2
        if mode=='dlin':
            mode_n=3
        if mode=='dlog':
            mode_n=4
        return self.write("WI%d,%d,%d,%+.3e,%+.3e,%d,%+.3e,%+.3e\n" % (ch, mode_n, range, start, stop, n, v_lim, p_lim))

    def sweep_v(self, ch, mode='lin', range=0, start=0, stop=1, n=50, i_lim=0.001, p_lim=1):
        self.mode="SWEEP"
        mode_n=1;
        if mode=='log':
            mode_n=2
        if mode=='dlin':
            mode_n=3
        if mode=='dlog':
            mode_n=4
        return self.write("WV%d,%d,%d,%+.3e,%+.3e,%d,%+.3e,%+.3e\n" % (ch, mode_n, range, start, stop, n, i_lim, p_lim))

    def sweep_i_follow(self, ch, range=0, start=0.001, stop=0.001, v_lim=1, p_lim=1):
        return self.write("WSI%d,%d,%+.3e,%+.3e,%+.3e,%+.3e"%(ch, range, start, stop, v_lim, p_lim))
        
    def sweep_v_follow(self, ch, range=0, start=0, stop=1, i_lim=0.001, p_lim=1):
        return self.write("WSV%d,%d,%+.3e,%+.3e,%+.3e,%+.3e"%(ch, range, start, stop, i_lim, p_lim))
    
    def sweep_timing(self, hold=0, delay=0):
        return self.write("WT%.3e,%3e" % (hold, delay))
    
    def search_cfg(self, op="FBpos", meas="searchV", itime=5e-3):
        operations={ 'FBpos': 1,
              'FBneg': 2,
              'ramp_gt': 3,
              'ramp_lt': 4 }
        
        measurements = {'searchV': 1,
                        'searchI': 2,
                        'searchVsenseVI': 3,
                        'searchIsenseVI': 4 }
        
        opcode=operations[op] or None
        mcode = measurements[meas] or None
        
        self.write("ASM%d,%d,%f" % (opcode, mcode, itime))
        
    def search(self, ch, start=0, stop=1, rate=500, i_lim=1e-3):
        G=max(abs(start), abs(stop))
        D=abs(start-stop)
        self.write("ASV%d,%f,%f,%f,%f"%(ch, start, stop, rate, i_lim))
        
    def search_timing(self,hold=0.0, delay=0.0):
        self.write("AT%f,%f"%(hold, delay))
        
    def sense_i(self, ch, v=1, i_target=1e-4, i_lim=1e-3):
        self.write("AVI%d,%f,%f,%f"%(ch, v, i_target, i_lim))
    
    def sense_v(self, ch, i=1e-3, v_target=0.1, v_lim=1):
        self.write("AIV%d,%f,%f,%f"%(ch, i, v_target, v_lim))
    
        
    def connect(self,ch):
        cmd=""
        if ch:
            if type(ch)==type([]):
                cmd = "CN" + (",".join(list(map(str,ch)))) + "\n"
            else:
                cmd = "CN%s\n" % ch
        else:
            cmd="CN\n"
        self.write(cmd)

    def zero(self, ch=None):
        cmd=""
        if ch:
            if type(ch)==type([]):
                cmd = "DZ" + (",".join(list(map(str,ch)))) + "\n"
            else:
                cmd = "DZ%s\n" % ch
        else:
            cmd="DZ\n"
        self.write(cmd)
       
    def calibrate(self):
        self.write("CA")
        time.sleep(10)
        self.status()
        print("IBSTA: "+ gpib_err(inst.ibsta()))

        print("Calibrated")
    
    def disconnect(self,ch=None):
        cmd=""
        if ch:
            if type(ch)==type([]):
                cmd = "CL" + (",".join(list(map(str,ch)))) + "\n"
            else:
                cmd = "CL%s\n" % ch
        else:
            cmd="CL\n"
        self.write(cmd)
        
    def opstat(self):
        self.write("LOP?")
        codestring = self.readresult()
        codestring = codestring.replace("\n", "").replace("\r", "")
        codes = codestring[3:].split(",")

        codetotext = { '00': 'Not active',
               '01': 'V source',
               '02': 'I+ source',
               '03': 'I- source',
               '10': 'Both VS limit',
               '11': 'V compliance reached',
               '12': 'I+ compliance reached',
               '13': 'I- compliance reached',
               '20': 'Oscillating',
               '30': 'HVU not settled'
              }
        
        return list(map(lambda c: codetotext[c], codes))
        
        

        
    def readresult(self):
        readmore=True
        outdata=""
        ibread=""
        self._resdata = ""
    
        while readmore:
            try:
                ibread=inst.read().decode('ascii')
                if ibread.endswith("\n"):
                    readmore=False
            except:
            
                readmore=False
                
            outdata=outdata+ibread

        #print("IBSTA: "+ gpib_err(inst.ibsta()))
        self._resdata = outdata
        return outdata

    def xe(self):
        return self.write("XE")

    
def parseresult(res, channelnames):
    results = {}
    
    errors_reported = []

    items = res.split(",")
    for i in items:
        st=i[0]
        ch=channelnames[int((ord(i[1])-ord('A')+1)/2-1)]
        meas=i[2]
        v=float(i[3:])
        if(st in ('N', 'W', 'E')):
           resname="%s_%s"%(meas, ch)
           resarr=results.get(resname, [])
           resarr.append(v)
           results[resname] = resarr
        else:
            if not(st in errors_reported):
                print("MEAS_ERR: %s" % i)
                errors_reported.append(st)
    return results


hp4142n_status_bits= { 
    'Data ready': 0,
    'Wait': 1,
    'NotUsed': 2,
    'Interlock open': 3,
    'Set ready': 4,
    'Error': 5,
    'RQS': 6,
    'Shut down': 7
    }

gpib_err_bits={
        'DCAS' : 0,
        'DTAS' : 1,
        'LACS' : 2,
        'TACS' : 3,
        'ATN' : 4,
        'CIC' : 5,
        'REM' : 6,
        'LOK' : 7,
        'Complete' : 8,
        'EVENT' : 9,
        'SPOLL' : 10,
        'RQS' : 11,
        'SRQI' : 12,
        'END' : 13,
        'TIMO' : 14,
        'ERR' : 15
};

def gpib_err(bits):
    errs = []
    for err in gpib_err_bits:
        if bits & (2**gpib_err_bits[err]):
            errs.append(err)

    return " ".join(errs)

def statusdecode(bits):
    stat = []
    for err in hp4142n_status_bits:
        if bits & (2**hp4142n_status_bits[err]):
            stat.append(err)

    return " ".join(stat)

In [58]:
channels=['X','Y','A']
A=6

r=None

inst = Gpib.Gpib(0, 2)
smu = hp4142b(inst)
#smu.calibrate()

def meas_diode():
    smu.connect([A])
    smu.sweep_i(A, start=1e-8, stop=100e-3, v_lim=20, mode='log', n=50)
    smu.write("MM2,6\n")

    smu.xe()
    smu.zero()
    smu.disconnect()

    r=smu.readresult()
    #print(r)

    d=parseresult(r, channels)

    return d

def meas_diode_rev(): 
    smu.connect([A])
    #smu.sweep_v(A, start=-0.1, stop=-10, i_lim=0.01, n=40)
    smu.sweep_i(A, start=-1e-8, stop=-40e-3, v_lim=-20, mode='log', n=50, p_lim=5)

    smu.write("MM2,6\n")
    smu.sweep_timing(hold=0.05, delay=0.05)
    smu.xe()
#    time.sleep(15)
    smu.zero()
    smu.disconnect()
    smu.status()

    r=smu.readresult()
    #print(r)
    d=parseresult(r, channels)

    return d

def vz(V_test=20, Iz=20e-3):    
    smu.connect([A])
    smu.current(A, -Iz, v_lim=V_test)
    smu.write("MM1,6\n")
    smu.xe()
    print(smu.opstat())
    smu.zero()
    smu.disconnect()

    r=smu.readresult()
    print(r)
    d=parseresult(r, channels)
    #print(d)
    return -d['V_A'][0]

#diode=meas_diode()
#rev=None
rev=meas_diode_rev()
i1=0.1e-3
vz1=vz(Iz=i1)
i2=35e-3
vz2=vz(Iz=i2)
Rz=(vz2-vz1)/(i2-i1)

print("Vz@%f mA = %f" % (1e3*i1, vz1))
print("Vz@%f mA = %f" % (1e3*i2, vz2))
print("Rz = %f"%(Rz))

print("Done")


HEWLETT PACKARD,4142B,0,4.30

Status: 0
 
['Not active', 'Not active', 'Not active', 'Not active', 'Not active', 'I- source', 'Not active', 'Not active']
NFV-09.7456E+00

['Not active', 'Not active', 'Not active', 'Not active', 'Not active', 'I- source', 'Not active', 'Not active']
NFV-10.0140E+00

Vz@0.100000 mA = 9.745600
Vz@35.000000 mA = 10.014000
Rz = 7.690544
Done


In [59]:

tracerev =  go.Scatter(x=np.array(rev['V_A']), y=np.array(rev['I_A']), name='Reverse')
tracefwd = go.Scatter(x=np.array(diode['V_A']), y=np.array(diode['I_A']), name="Forward")
tracevz = go.Scatter(x=[-vz2, -vz1], y=[-i2, -i1], name='Rz=%f' % Rz)

data=[ tracefwd, tracerev, tracevz]
#data= [tracerev]
layout = go.Layout(
    title='Zener diode forward drop',
    xaxis=dict(
        title='V',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#000000'
        )
    ),
    yaxis=dict(
        title='I',
        #type='log',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#000000'
        )
    )
)


iplot({ 'data': data, "layout": layout }) 
    